In [1]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import img_to_array, load_img
# from keras_preprocessing.image import img_to_array, load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.applications.densenet import DenseNet201, preprocess_input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from shutil import copyfile

from sklearn.model_selection import train_test_split
import cv2
import tensorflow.keras.utils as image
import shutil

In [2]:
image_dir = './data/'
with_mask_dir = './trained_data/with_mask'
without_mask_dir = './trained_data/without_mask'

In [3]:
df = pd.read_csv('./train_labels.csv')

In [4]:
# for index, row in df.iterrows():
#     filename = row['image']
#     label = row['target']
#     if label == 0:
#         dest_dir = without_mask_dir
#     else:
#         dest_dir = with_mask_dir
#     if not os.path.exists(dest_dir):
#         os.makedirs(dest_dir)
#     src_path = os.path.join(image_dir, filename)
#     dest_path = os.path.join(dest_dir, filename)
#     copyfile(src_path, dest_path)

In [5]:
dataset_path = './trained_data/'
img_size = (224, 224)
batch_size = 32

In [6]:
data_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [7]:
train_generator = data_generator.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary')

Found 1308 images belonging to 2 classes.


In [8]:
input_shape = (224, 224, 3)
batch_size = 32

In [9]:
image_paths = []
labels = []

for subdir, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith('.jpg'):
            image_path = os.path.join(subdir, file)
            if 'with_mask' in subdir:
                label = 1
            elif 'without_mask' in subdir:
                label = 0
            else:
                continue # Skip images in other directories
            image_paths.append(image_path)
            labels.append(label)
            
labels = [str(label) for label in labels]
            

In [10]:
train_image_paths, val_image_paths, train_labels, val_labels = train_test_split(image_paths, labels, test_size=0.2)


In [11]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
    )

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [12]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'image': train_image_paths, 'label': train_labels}),
    x_col='image',
    y_col='label',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    )

val_generator = val_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'image': val_image_paths, 'label': val_labels}),
    x_col='image',
    y_col='label',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    )

Found 985 validated image filenames belonging to 2 classes.
Found 247 validated image filenames belonging to 2 classes.


In [13]:
desnet201 = DenseNet201(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))


In [25]:
x = desnet201.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=desnet201.input, outputs=predictions)

for layer in model.layers[:300]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


In [26]:
train_steps = len(train_image_paths) 
val_steps = len(val_image_paths) 

In [27]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=15,
    validation_data=val_generator,
    validation_steps=len(val_generator)
    )

Epoch 1/15
31/31 [==============================] - 133s 3s/step - loss: 0.4123 - accuracy: 0.8081 - val_loss: 0.1477 - val_accuracy: 0.9433
Epoch 2/15
31/31 [==============================] - 83s 3s/step - loss: 0.1472 - accuracy: 0.9431 - val_loss: 0.1542 - val_accuracy: 0.9636
Epoch 3/15
31/31 [==============================] - 88s 3s/step - loss: 0.1109 - accuracy: 0.9655 - val_loss: 0.1856 - val_accuracy: 0.9555
Epoch 4/15
31/31 [==============================] - 88s 3s/step - loss: 0.0777 - accuracy: 0.9756 - val_loss: 0.1476 - val_accuracy: 0.9514
Epoch 5/15
31/31 [==============================] - 95s 3s/step - loss: 0.0377 - accuracy: 0.9888 - val_loss: 0.1567 - val_accuracy: 0.9555
Epoch 6/15
31/31 [==============================] - 138s 4s/step - loss: 0.0386 - accuracy: 0.9848 - val_loss: 0.1217 - val_accuracy: 0.9676
Epoch 7/15
31/31 [==============================] - 105s 3s/step - loss: 0.0314 - accuracy: 0.9919 - val_loss: 0.2622 - val_accuracy: 0.9393
Epoch 8/15
31/31 

In [28]:
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('Model accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Val'], loc='upper left')
# plt.show()

# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Val'], loc='upper left')
# plt.show()

In [29]:
# model.save('my_model.h5')

In [30]:

# def predict_mask(image_path):
#     img = image.load_img(image_path, target_size=(224, 224))
#     img_array = image.img_to_array(img)
#     img_batch = np.expand_dims(img_array, axis=0)
#     img_preprocessed = img_batch / 255.

#     prediction = model.predict(img_preprocessed)
#     if prediction < 0.5:
#         print('No mask')
#     else:
#         print('Mask')

#     plt.imshow(img)
#     plt.show()

# image_path = './Trials/img3.jpg'
# predict_mask(image_path)

In [31]:
# import keras.utils as image

# image_dir = './images/'

# predicted_classes = []

# for filename in os.listdir(image_dir):
    # img_path = os.path.join(image_dir, filename)
    # img = image.load_img(img_path, target_size=(224, 224))
    # x = image.img_to_array(img)
    # x = np.expand_dims(x, axis=0)
    # x = preprocess_input(x)
    # prediction = model.predict(x)

#     if prediction < 0.5:
#         print('{}: No mask'.format(filename))
#     else:
#         print('{}: Mask'.format(filename))

In [32]:
image_list = pd.read_csv('./SampleSubmission.csv')
output_df = pd.DataFrame(columns=['image', 'target'])
i_dir = './images/'

for i in range(len(image_list)):
    filename = image_list['image'][i]
    img_path = os.path.join(i_dir, filename)
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    prediction = model.predict(x)
    prediction = np.round(prediction)[0][0]
    # prediction = round(float(prediction), 3)
    output_df.loc[i] = [filename, prediction]
    output_df.columns = ['id', 'label']

output_df.to_csv('Desnet201v3.csv', index=False)

In [33]:
# def move_images_from_csv(src_dir, dst_dir, csv_path):
#     df = pd.read_csv(csv_path)
    
#     os.makedirs(dst_dir, exist_ok=True)
    
#     for filename in os.listdir(src_dir):
#         if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
#             if filename in df['image'].values:
#                 src_path = os.path.join(src_dir, filename)
#                 dst_path = os.path.join(dst_dir, filename)
                
#                 shutil.move(src_path, dst_path)

# dst_dir = './validation_data'
# src_dir = './images/'
# csv_path = './SampleSubmission.csv'

# move_images_from_csv(src_dir, dst_dir, csv_path)

In [34]:
# def display_image_with_prediction(csv_path, img_dir, idx):
#     df = pd.read_csv(csv_path)
    
#     filename = df.iloc[idx]['id']
#     prediction = df.iloc[idx]['label']
    
#     path = os.path.join(img_dir, filename)
#     img = cv2.imread(path)
    
#     if prediction == 0:
#         title = "No Mask"
#     else:
#         title = "Mask"
    
#     cv2.imshow(title, img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    
# dst_dir = './validation_data'
# csv_path = './ResNet50.csv'

# display_image_with_prediction(csv_path, dst_dir, 506)

In [41]:
df1 = pd.read_csv('./Submission7.csv')
df2 = pd.read_csv('./Submission9.csv')

merged_csv = pd.merge(df1, df2, on='id', suffixes=('_1', '_2'))

different_labels = merged_csv[merged_csv['label_1'] != merged_csv['label_2']]

different_labels.to_csv('different_labels.csv', index=False)